![MLU Logo](../data/MLU_Logo.png)

# <a name="0">Machine Learning Accelerator - Natural Language Processing - Lecture 1</a>

## Final Project: K Nearest Neighbors Model for the IMDB Movie Review Dataset

For the final project, build a K Nearest Neighbors model to predict the sentiment (positive or negative) of movie reviews. The dataset is originally hosted here: http://ai.stanford.edu/~amaas/data/sentiment/

Use the notebooks from the class and implement the model, train and test with the corresponding datasets.

**You can follow these steps**
1. Read training-test data (Given)
2. Train a KNN classifier (Implement)
3. Make predictions on your test dataset (Implement):


__You can use the KNN Classifier from here: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html__

- ### Importing

In [2]:
import pandas as pd
import re
# nltk 
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer, WordNetLemmatizer

# word vectorizers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# modeling ...
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

## 1. Reading the dataset

We will use the __pandas__ library to read our dataset. 

#### __Training data:__
Let's read our training data. Here, we have the text and label fields. Labe is 1 for positive reviews and 0 for negative reviews.

In [3]:
train_df = pd.read_csv('../data/final_project/imdb_train.csv', header=0)
print('Training shape : ', train_df.shape)
train_df.head()

Training shape :  (25000, 2)


,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


## 2. Train a KNN Classifier
Here, you will apply pre-processing operations we covered in the class. Then, you can split your dataset to training and validation here. For your first submission, you will use __K Nearest Neighbors Classifier__. It is available [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

- ### Clean Text

In [4]:
# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

# Stemmer
snow = SnowballStemmer('english')

def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""
            
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:

        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words
 
        final_text_list.append(final_string)
        
    return final_text_list

    

In [5]:
print("Processing the reviewText fields")
train_text_list = process_text(train_df["text"].tolist())

Processing the reviewText fields


In [6]:
len(train_text_list) == len(train_df["text"])

True

In [7]:
# create word vector

tf = TfidfVectorizer(max_features=50)
tf.fit(train_text_list)

# print(tf.vocabulary_)
X_train = tf.transform(train_text_list).toarray()
y_train = train_df['label']

In [8]:
X_train.shape

(25000, 50)

In [9]:
y_train.shape

(25000,)

In [10]:
knn = KNeighborsClassifier()

In [11]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

## 3. Make predictions on your test dataset

Once we select our best performing model, we can use it to make predictions on the test dataset. You can simply use __.fit()__ function with your training data to use the best performing K value and use __.predict()__ with your test data to get your test predictions.

- ### Prepare Test Data

In [12]:
test_df = pd.read_csv('../data/final_project/imdb_test.csv', header=0)
print('Test shape : ', test_df.shape)
test_df.head()

Test shape :  (25000, 2)


,text,label
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


In [13]:
print("Processing the reviewText fields")
test_text_list = process_text(test_df["text"].tolist())

Processing the reviewText fields


In [14]:
# create word vector

tf = TfidfVectorizer(max_features=50)
tf.fit(test_text_list)

# print(tf.vocabulary_)
X_test = tf.transform(test_text_list).toarray()
y_test = test_df['label']

print('Test text shape : ', X_test.shape)
print('Test label shape : ', y_test.shape)

Test text shape :  (25000, 50)
Test label shape :  (25000,)


In [15]:
y_pred = knn.predict(X_test)

C:\Users\USER\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## Results

In [16]:
cm = confusion_matrix(y_test, y_pred)

In [17]:
cm

array([[8162, 4338],
       [4402, 8098]], dtype=int64)

In [18]:
accuracy_score(y_test, y_pred)

0.6504

In [19]:
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.65      0.65      0.65     12500\n           1       0.65      0.65      0.65     12500\n\n    accuracy                           0.65     25000\n   macro avg       0.65      0.65      0.65     25000\nweighted avg       0.65      0.65      0.65     25000\n'